# LangChain with networkx

Date: 2024/08/09

Reference: https://python.langchain.com/v0.1/docs/integrations/graphs/networkx/

In [1]:
from langchain.indexes import GraphIndexCreator
from langchain_openai import OpenAI

In [2]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [3]:
with open("doc/state_of_the_union.txt") as f:
    all_text = f.read()

In [4]:
text = "\n".join(all_text.split("\n\n")[105:108])
text

'It won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built. \nThis is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. '

In [5]:
graph = index_creator.from_text(text)

In [6]:
graph.get_triples()

[('Intel', '$20 billion semiconductor “mega site”', 'is going to build'),
 ('Intel', 'Silicon Valley', 'helped build'),
 ('Intel', 'up to eight state-of-the-art factories', 'is going to build'),
 ('Intel', '10,000 new good-paying jobs', 'is going to create'),
 ('America', '“Field of dreams”', 'will be built on')]

In [7]:
from langchain.chains import GraphQAChain

In [8]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [9]:
chain.run("what is Intel going to build?")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
 Intel
Full Context:
Intel is going to build $20 billion semiconductor “mega site”
Intel helped build Silicon Valley
Intel is going to build up to eight state-of-the-art factories
Intel is going to create 10,000 new good-paying jobs

> Finished chain.


' Intel is going to build a $20 billion semiconductor "mega site" and up to eight state-of-the-art factories.'